In [2]:
from chess_nn import ChessNN
import torch
import torch.nn.functional as F
from chess_env import ChessEnv, state_to_alpha_zero_input
from chess_nn import ChessNN
from torch import optim
from tree_search import MCTS
from tree_search import Node
from helpers import move_probabilities_to_policy, policy_to_move_probabilities
from train import play_game, store_experiences

LEARNING_RATE = 0.001
chess_nn = ChessNN()
optimizer = optim.Adam(chess_nn.parameters(), lr=LEARNING_RATE)
experiences = []

episode_reward, mcts_white = play_game(chess_nn)


 r n b q k b n r
p p p p p p p p
. . . . . . . .
. . . . . . . .
. . . . . . . .
. . . . . . . .
P P P P P P P P
R N B Q K B N R
g1h3 


 r n b q k b n r
p p p p p p p p
. . . . . . . .
. . . . . . . .
. . . . . . . .
. . . . . . . N
P P P P P P P P
R N B Q K B . R
h7h5 


 r n b q k b n r
p p p p p p p .
. . . . . . . .
. . . . . . . p
. . . . . . . .
. . . . . . . N
P P P P P P P P
R N B Q K B . R
f2f3 


 r n b q k b n r
p p p p p p p .
. . . . . . . .
. . . . . . . p
. . . . . . . .
. . . . . P . N
P P P P P . P P
R N B Q K B . R
g7g5 


 r n b q k b n r
p p p p p p . .
. . . . . . . .
. . . . . . p p
. . . . . . . .
. . . . . P . N
P P P P P . P P
R N B Q K B . R
h1g1 


 r n b q k b n r
p p p p p p . .
. . . . . . . .
. . . . . . p p
. . . . . . . .
. . . . . P . N
P P P P P . P P
R N B Q K B R .
h8h6 


 r n b q k b n .
p p p p p p . .
. . . . . . . r
. . . . . . p p
. . . . . . . .
. . . . . P . N
P P P P P . P P
R N B Q K B R .
c2c4 


 r n b q k b n .
p p p p p p . .
. . . .

In [15]:
store_experiences(mcts_white.current_node, episode_reward, experiences)
# Passes a list of size (num_experiences, 19, 8, 8) to chess_nn
predicted_value, predicted_policy = chess_nn(torch.stack([state_to_alpha_zero_input(node.board_state) for node in experiences]))
target_value = torch.tensor([node.reward for node in experiences]).unsqueeze(1).float()
target_policy = torch.tensor([move_probabilities_to_policy(node.get_move_probabilities()) for node in experiences]).float()
print(predicted_policy.dtype)
print(target_policy.dtype)
loss_value = F.mse_loss(predicted_value, target_value)
loss_policy = F.mse_loss(predicted_policy, target_policy)
loss = loss_value + loss_policy

optimizer.zero_grad()
loss.backward()
optimizer.step()

/Users/shivamjindal/Documents/WATai/RL-Chess-Engine/chess_nn.py:22: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  output = self.soft_max(output)


torch.float32
torch.float32
